## Part 2 of a 3-part assignment for Week 3:  on Toronto Neighborhood
- ### Dataframe with latitude and longitude coordinates for the neighorhood from selected postal codes
- #### 1. Could not succeed in getting response from the package geocoder.
- #### 2. Accessed using pd.read_csv file
- #### 3. Used the codes and information on the sorted dataframe of the Part 1. Thus Notebook cells have been retained.
- #### 4. First few cells are same as in Part 1 of the assignment.

#### Import libraries and install Beautifulsoup4

In [1]:
import pandas as pd
import sys
sys.path.append('C:\\Python27\\Scripts\\pip.exe')
!pip install beautifulsoup4

!pip install lxml
!pip install html5lib
!pip install requests

from bs4 import BeautifulSoup
import requests
import urllib.request
from lxml import html

#### Scrape the web page containing Canadian postal codes, boroughs and neighborhoods

In [2]:
# Scrape the page with BeautifulSoup

quote_page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(quote_page)
#print(page.status_code)

soup= BeautifulSoup(page.text, "html.parser")

right_table=soup.find('table', {"class" : 'wikitable sortable'})
#print(right_table)

# Transfer to a DataFrame 
df = pd.read_html(str(right_table))[0]
df_all = pd.DataFrame(df)
#print(df_all.shape)
#df_all.head(5)

#### Get the dataframe after filtering out "Not assigned" boroughs and neighborhoods etc., restricting in the postal codes selected, and concatenating the neighborhoods under the same borough/postal code  

In [3]:
# Data shaping 

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element) +","
    return result[:-1]
#print(concatenate_list_data(['a','c']))

df.drop(  df[ df['Borough']=='Not assigned'].index, inplace=True)
restr = df['Neighborhood']=='Not assigned'
df.replace(to_replace=list(df[restr]['Neighborhood']), \
                 value= list(df[restr]['Borough']), inplace=True)
df_filtered = df
print("Size for filtered data table: ", df_filtered.shape)

post_lst=['M5G','M2H', 'M4B','M1J','M4G','M4M', 'M1R','M9V','M9L','M5V','M1B','M5A']

# print(df[df['Postcode'].isin(post_lst)])


newdf=pd.DataFrame(columns=['Postcode','Borough','Neighborhood'])

for pst in post_lst:
    restr= df['Postcode']==pst
    neigh=df[ restr]['Neighborhood'].to_list()
    neigh=concatenate_list_data(neigh)
    boro=df[restr]['Borough'].to_list()[0]
    addrows={'Postcode':pst,'Borough':boro,'Neighborhood':neigh}
    newdf=newdf.append(addrows, ignore_index=True)

#print("Size for the selected 12 postalcode data: ", newdf.shape)
#newdf

Size for filtered data table:  (210, 3)


#### Retrival of Longitude and Latitude for the postal code areas


In [4]:
# This part did not work 

# import geocoder # import geocoder
# postal_code = 'M5A'
# initialize your variable to None
# lat_lng_coords = None
# loop until you get the coordinates
# while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng
# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]


In [5]:
# Part 2 
path='http://cocl.us/Geospatial_data'
df_geosp = pd.read_csv(path)
df_geosp.head()    
custom_list=list(newdf['Postcode'])
df_geo1=df_geosp[df_geosp['Postal Code'].isin(custom_list)]

#print(custom_list)
#print(df_geo1)
df_geo1['Postal Code']=pd.Categorical(df_geo1['Postal Code'], custom_list )
df_geo2=df_geo1.sort_values('Postal Code').reset_index(drop=True)

df_geo2[['Borough','Neighborhood']]=newdf[['Borough','Neighborhood']]
df_geo2[['Postal Code','Borough','Neighborhood','Latitude', 'Longitude']]


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale,Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420


## Thanks
## END of Part 2